
### Initialization Functions


In [ ]:
!mkdir dayavgdata


In [ ]:
!mkdir avgsdata

In [ ]:
!rm *.nc


rm: cannot remove '*.nc': No such file or directory


In [ ]:
!mv ./avgsdata/* ./

Zips files in dayavgdata folder

In [ ]:
import zipfile
import os

# Define the folder path
folder_path = "./dayavgdata"

# Define the zip file name
zip_file_name = "dayavgdata.zip"

# Create a ZipFile object
zip_file = zipfile.ZipFile(zip_file_name, mode="w")

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    # Get the full path of the file
    file_path = os.path.join(folder_path, file_name)

    # Add the file to the zip file
    zip_file.write(file_path, file_name)

# Close the zip file
zip_file.close()

print(f"All files in {folder_path} have been zipped to {zip_file_name}")


All files in ./dayavgdata have been zipped to dayavgdata.zip


Remove all files from "avg" folder

### Get data and save averaage functions

In [ ]:
!pip install netCDF4

In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import glob
import requests
import os

def take_dayavg(urls):

  for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
        print(f"{filename} download done")
  print("all urls done")

  # open all the .nc files in the folder
  files = glob.glob("*.nc")

  file1 = files[0]
  avg_file_name = file1.split("_")[:-4]
  avg_file_name = "_".join(avg_file_name)
  avg_file_name = "dayavg_" + avg_file_name + ".nc"
  avg_file_name = "dayavgdata/" + avg_file_name


  if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")

  # take total time_size from all the files
  time_size = 0
  for file in files:
      ds = nc.Dataset(file)
      time_size += ds["zmla"].shape[0]

  # create a new .nc file to store the average zmla daywise
  ds_temp = nc.Dataset(file1)
  lat_len = ds_temp["zmla"].shape[1]
  lon_len = ds_temp["zmla"].shape[2]
  ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
  ds_out.createDimension("time", time_size)
  ds_out.createDimension("lat", lat_len)
  ds_out.createDimension("lon", lon_len)
  ds_out.createVariable("zmla", "f4", ("time", "lat", "lon"))
  ds_out.variables["zmla"][:] = 0
  ds_out.close()

  time_done = 0
  # loop over all the files
  for file in files:
      ds = nc.Dataset(file)
      # read zmla data
      zmla = ds["zmla"][:, :, :]
      # print(zmla.shape)
      # add  the zmla data to the new .nc file use time_size to keep track of the time
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      # use time_done to keep track of the time
      ds_out.variables["zmla"][time_done:time_done + zmla.shape[0], :, :] = zmla[:, :, :]
      ds_out.close()

  !find . -maxdepth 1 -type f -name '*.nc' ! -name 'avg_*' -delete

  print("done!")

def combine_mult_files():
  files = glob.glob("*.nc")

  file1 = files[0]
  avg_file_name = file1.split("_")[:3]
  avg_file_name = "_".join(avg_file_name)
  avg_file_name = "dayavg_" + avg_file_name + ".nc"
  avg_file_name = "dayavgdata/" + avg_file_name


  if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")

  # take total time_size from all the files
  time_size = 0
  for file in files:
      ds = nc.Dataset(file)
      time_size += ds["zmla"].shape[0]

  # create a new .nc file to store the average zmla daywise
  ds_temp = nc.Dataset(file1)
  lat_len = ds_temp["zmla"].shape[1]
  lon_len = ds_temp["zmla"].shape[2]
  ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
  ds_out.createDimension("time", time_size)
  ds_out.createDimension("lat", lat_len)
  ds_out.createDimension("lon", lon_len)
  ds_out.createVariable("zmla", "f4", ("time", "lat", "lon"))
  ds_out.variables["zmla"][:] = 0
  ds_out.close()

  time_done = 0
  # loop over all the files
  for file in files:
      ds = nc.Dataset(file)
      # read zmla data
      zmla = ds["zmla"][:, :, :]
      # print(zmla.shape)
      # add  the zmla data to the new .nc file use time_size to keep track of the time
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      # use time_done to keep track of the time
      ds_out.variables["zmla"][time_done:time_done + zmla.shape[0], :, :] = zmla[:, :, :]
      ds_out.close()

  !find . -maxdepth 1 -type f -name '*.nc' ! -name 'avg_*' -delete

  print("done!")




def take_dayavg_multiple(urls):
  n = len(urls)
  m = len(urls[0])

  for i in range(m):
    for j in range(n):
      # print(urls[j][i])
      response = requests.get(urls[j][i])
      filename = urls[j][i].split('/')[-1]
      with open(filename, 'wb') as f:
        f.write(response.content)
        print(f"{filename} download done")


    files = glob.glob("*.nc")

    file1 = files[0]
    avg_file_name1 = file1.split('_')[:3]
    avg_file_name2 = file1.split('_')[-1]
    avg_file_name = '_'.join(avg_file_name1) + "_"+ avg_file_name2
    avg_file_name = "avgsdata/" + avg_file_name
    print(avg_file_name)

    if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")

    ds_temp = nc.Dataset(file1)
    time_len = ds_temp["zmla"].shape[0]
    lat_len = ds_temp["zmla"].shape[1]
    lon_len = ds_temp["zmla"].shape[2]
    ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
    ds_out.createDimension("time", time_len)
    ds_out.createDimension("lat", lat_len)
    ds_out.createDimension("lon", lon_len)
    ds_out.createVariable("zmla", "f4", ("time", "lat", "lon"))
    ds_out.variables["zmla"][:] = np.zeros((time_len, lat_len, lon_len))
    ds_out.close()

    for file in files:
      ds = nc.Dataset(file)
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      ds_out.variables["zmla"][:] += ds["zmla"][:]
      ds_out.close()

    ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
    ds_out.variables["zmla"][:] /= len(files)
    ds_out.close()

    !rm *.nc

  !mv ./avgsdata/* ./
  #
  #
  # now combine all files
  #
  #
  files = glob.glob("*.nc")

  file1 = files[0]
  avg_file_name = file1.split("_")[:3]
  avg_file_name = "_".join(avg_file_name)
  avg_file_name = "dayavg_" + avg_file_name + ".nc"
  avg_file_name = "dayavgdata/" + avg_file_name


  if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")

  # take total time_size from all the files
  time_size = 0
  for file in files:
      ds = nc.Dataset(file)
      time_size += ds["zmla"].shape[0]

  # create a new .nc file to store the average zmla daywise
  ds_temp = nc.Dataset(file1)
  lat_len = ds_temp["zmla"].shape[1]
  lon_len = ds_temp["zmla"].shape[2]
  ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
  ds_out.createDimension("time", time_size)
  ds_out.createDimension("lat", lat_len)
  ds_out.createDimension("lon", lon_len)
  ds_out.createVariable("zmla", "f4", ("time", "lat", "lon"))
  ds_out.variables["zmla"][:] = 0
  ds_out.close()

  time_done = 0
  # loop over all the files
  for file in files:
      ds = nc.Dataset(file)
      # read zmla data
      zmla = ds["zmla"][:, :, :]
      # print(zmla.shape)
      # add  the zmla data to the new .nc file use time_size to keep track of the time
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      # use time_done to keep track of the time
      ds_out.variables["zmla"][time_done:time_done + zmla.shape[0], :, :] = zmla[:, :, :]
      ds_out.close()

  !find . -maxdepth 1 -type f -name '*.nc' ! -name 'avg_*' -delete

  print("done!")





### Average of data day wise


In [ ]:
# INM-CM4-8 ! 7

urls81 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

take_dayavg(urls81)


zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc download done
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc download done
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc download done
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc download done
all urls done
done!


In [ ]:
# INM-CM5-0 ! 8

urls91 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

take_dayavg(urls91)

zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19790101-19881231.nc download done
zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19890101-19981231.nc download done
zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19990101-20081231.nc download done
zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_20090101-20141231.nc download done
all urls done
done!


In [ ]:
# IPSL-CM6A-LR ! 9

urls101 = [
    "http://vesg.ipsl.upmc.fr/thredds/fileServer/cmip6/CMIP/IPSL/IPSL-CM6A-LR/amip/r2i1p1f1/Eday/zmla/gr/v20181109/zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc"
]

take_dayavg(urls101)

zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc download done
all urls done
done!


In [ ]:
# MIROC-ES2L ! 10

urls111 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]


urls112 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls119 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls11 = [urls111,urls112,urls119]

take_dayavg_multiple(urls11)

In [ ]:
# CESM2 FV2 ! 2

urls21 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls22 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]
urls23 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls24 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20100101-20150103.nc"
    ]

urls2 = [urls21,urls22,urls23]

take_dayavg_multiple(urls2)

zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc download done
avgsdata/zmla_Eday_CESM2-FV2_19500101-19591231.nc


OSError: ignored

In [ ]:
# CESM2 WACCM ! 3
urls31 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc"
    ]
urls32 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls33 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls3 = [urls31,urls32,urls33]
take_dayavg_multiple(urls3)

In [ ]:
combine_mult_files()

done!


### Do all *avgs* for data model wise

In [ ]:
# CESM2 ! 1
urls1 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190326/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]


urls3 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls4 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20100101-20150101.nc"
    ]

urls5 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20100101-20150101.nc"
    ]
urls6 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20100101-20150101.nc"
    ]
urls7 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20100101-20150101.nc"
    ]
urls8 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20100101-20150101.nc"
    ]
urls9 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20100101-20150101.nc"
    ]
urls10 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20100101-20150101.nc"
    ]


take_avg_zmla(urls1)
take_avg_zmla(urls2)


In [ ]:
# CESM2 FV2 ! 2

urls21 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls22 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]
urls23 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls24 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20100101-20150103.nc"
    ]

take_avg_zmla(urls21)
take_avg_zmla(urls22)
take_avg_zmla(urls23)
take_avg_zmla(urls24)

In [ ]:
# CESM2 WACCM ! 3
urls31 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc"
    ]
urls32 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls33 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

take_avg_zmla(urls31)
take_avg_zmla(urls32)
take_avg_zmla(urls33)

In [ ]:
# CESM WACCM FV2 4
urls41 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]

urls42 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls43 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

# take_avg_zmla(urls41)
# take_avg_zmla(urls42)
# take_avg_zmla(urls43)



zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc download done
all urls done


OSError: ignored

In [ ]:
# EC-Earth3-AerChem ! 5
urls51 = [
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19790101-19791231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19800101-19801231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19810101-19811231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19820101-19821231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19830101-19831231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19840101-19841231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19850101-19851231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19860101-19861231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19870101-19871231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19880101-19881231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19890101-19891231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19900101-19901231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19910101-19911231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19920101-19921231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19930101-19931231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19940101-19941231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19950101-19951231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19960101-19961231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19970101-19971231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19980101-19981231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19990101-19991231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20000101-20001231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20010101-20011231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20020101-20021231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20030101-20031231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20040101-20041231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20050101-20051231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20060101-20061231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20070101-20071231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20080101-20081231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20090101-20091231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20100101-20101231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20110101-20111231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20120101-20121231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20130101-20131231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20140101-20141231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20150101-20151231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20160101-20161231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20170101-20171231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20180101-20181231.nc"
    ]


urls52 = [
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19790101-19791231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19800101-19801231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19810101-19811231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19820101-19821231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19830101-19831231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19840101-19841231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19850101-19851231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19860101-19861231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19870101-19871231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19880101-19881231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19890101-19891231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19900101-19901231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19910101-19911231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19920101-19921231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19930101-19931231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19940101-19941231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19950101-19951231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19960101-19961231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19970101-19971231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19980101-19981231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19990101-19991231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20000101-20001231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20010101-20011231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20020101-20021231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20030101-20031231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20040101-20041231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20050101-20051231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20060101-20061231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20070101-20071231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20080101-20081231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20090101-20091231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20100101-20101231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20110101-20111231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20120101-20121231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20130101-20131231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20140101-20141231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20150101-20151231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20160101-20161231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20170101-20171231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20180101-20181231.nc"
    ]


take_avg_maxpblz(urls51)
take_avg_minpblz(urls52)

In [ ]:
# EC-Earth3-Veg-LR ! 6
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19790101-19791231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19800101-19801231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19810101-19811231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19820101-19821231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19830101-19831231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19840101-19841231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19850101-19851231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19860101-19861231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19870101-19871231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19880101-19881231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19890101-19891231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19900101-19901231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19910101-19911231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19920101-19921231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19930101-19931231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19940101-19941231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19950101-19951231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19960101-19961231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19970101-19971231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19980101-19981231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19990101-19991231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20000101-20001231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20010101-20011231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20020101-20021231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20030101-20031231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20040101-20041231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20050101-20051231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20060101-20061231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20070101-20071231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20080101-20081231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20090101-20091231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20100101-20101231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20110101-20111231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20120101-20121231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20130101-20131231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20140101-20141231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20150101-20151231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20160101-20161231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20170101-20171231.nc"
    ]

# take_avg_zmla(urls61)


In [ ]:
# INM-CM4-8 ! 7

urls81 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

take_avg_zmla(urls81)

In [ ]:
# INM-CM5-0 ! 8

urls91 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

take_avg_zmla(urls91)

In [ ]:
# IPSL-CM6A-LR ! 9

urls101 = [
    "http://vesg.ipsl.upmc.fr/thredds/fileServer/cmip6/CMIP/IPSL/IPSL-CM6A-LR/amip/r2i1p1f1/Eday/zmla/gr/v20181109/zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc"
]

take_avg_zmla(urls101)


In [ ]:
# MIROC-ES2L ! 10

urls111 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls112 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls113 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls114 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls115 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls116 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls117 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls118 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]

urls119 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]

take_avg_zmla(urls111)
take_avg_zmla(urls112)
take_avg_maxpblz(urls113)
take_avg_minpblz(urls114)
take_avg_maxpblz(urls115)
take_avg_minpblz(urls116)
take_avg_maxpblz(urls117)
take_avg_minpblz(urls118)
take_avg_zmla(urls119)


In [ ]:
# MPI-ESM-1-2-HAM ! 11

urls121 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19990101-20141231.nc"
]

urls122 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19990101-20141231.nc"
]

urls123 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19990101-20141231.nc"
]

urls124 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19990101-20141231.nc"
]

urls125 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19990101-20141231.nc"
]

urls126 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19990101-20141231.nc"
]

take_avg_maxpblz(urls121)
take_avg_minpblz(urls122)
take_avg_maxpblz(urls123)
take_avg_minpblz(urls124)
take_avg_maxpblz(urls125)
take_avg_minpblz(urls126)

### Analysis


In [ ]:
# open .nc file

import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import sys
import datetime
import scipy.stats as stats


# open file
f = nc.Dataset('/content/dayavg_zmla_Eday_INM-CM5-0.nc', 'r')
ablh = f.variables['zmla'][:]

print(np.shape(ablh))

# get lat and lon length
lat_length = ablh.shape[1]
lon_length = ablh.shape[2]


# robust trend function
def robust_trend(x, y):
    """
    Calculate the slope of a robust regression line.
    """
    return stats.theilslopes(y, x)[0]

# create empty array to store trend values
ablh_trend = np.empty_like(ablh[0,:,:])

# loop through lat and lon indices and estimate trend at each grid point
for i in range(lat_length):
    for j in range(lon_length):
        ablh_grid = ablh[:, i, j]

        # remove missing values
        # ablh_grid = ablh_grid[~np.isnan(ablh_grid)]

        # calculate the trend
        slope = robust_trend(np.arange(ablh.shape[0]), ablh_grid)
        # slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(ablh.shape[0]), ablh[:,i,j])
        ablh_trend[i,j] = slope
        print(i, j, slope)

print(np.shape(ablh_trend))

# plot the trend
plt.imshow(ablh_trend)
plt.colorbar()
plt.show()

